# SQL Project

Due to the increase in book reading the number of people using book apps has increased.
Our goal is to generate enough information to provide a basis for building a new product related to the field of books apps 

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
books = '''
        SELECT * FROM BOOKS
        '''

books = pd.io.sql.read_sql(books, con = engine)        

In [4]:
authors = '''
        SELECT * FROM authors

        '''
authors = pd.io.sql.read_sql(authors, con = engine)


In [5]:
publishers =  '''
        SELECT * FROM publishers

        '''
publishers = pd.io.sql.read_sql(publishers, con = engine)


In [6]:
ratings =  '''
        SELECT * FROM ratings

        '''
ratings = pd.io.sql.read_sql(ratings, con = engine)


In [7]:
reviews =  '''
        SELECT * FROM reviews

        '''
reviews = pd.io.sql.read_sql(reviews, con = engine)


In [8]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [9]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [10]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [11]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Find the number of books released after January 1, 2000.

In [13]:
query_1 =   '''
                    SELECT
                        COUNT(book_id) AS book_cnt
                    FROM
                        books
                    WHERE  
                        publication_date::date > '2000-01-01'
                    ''' 
query_1 = pd.io.sql.read_sql(query_1, con = engine)        
print(query_1)

   book_cnt
0       819


There are 819 books released after 2000

## Find the number of user reviews and the average rating for each book.

In [14]:
query_2 =    '''
                        SELECT
                            books.book_id AS book_id, 
                            COUNT(reviews.review_id) AS cnt_review,
                            AVG(ratings.rating) AS avg_rating
                        FROM
                            books
                        LEFT JOIN ratings ON ratings.book_id = books.book_id
                        LEFT JOIN reviews ON reviews.book_id = books.book_id
                        GROUP BY
                            books.book_id
                        ORDER BY
                            books.book_id
                        ''' 

query_2 = pd.io.sql.read_sql(query_2, con = engine)        
print(query_2)

     book_id  cnt_review  avg_rating
0          1           6    3.666667
1          2           2    2.500000
2          3           9    4.666667
3          4           4    4.500000
4          5          24    4.000000
..       ...         ...         ...
995      996           9    3.666667
996      997          15    3.400000
997      998          20    3.200000
998      999           4    4.500000
999     1000          24    3.833333

[1000 rows x 3 columns]


For each book we see the number of review and the average rating. 

## Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [15]:
query_3 = '''
        SELECT
            publishers.publisher_id AS publisher_id,
            publishers.publisher AS publisher_name,
            COUNT(DISTINCT(books.book_id)) AS cnt_books
        FROM
            publishers
        INNER JOIN books ON books.publisher_id = publishers.publisher_id
        WHERE
            books.num_pages > 50
        GROUP BY
            publishers.publisher_id, publishers.publisher
        ORDER BY
            cnt_books DESC
        LIMIT 20;
            
    
'''
query_3 = pd.io.sql.read_sql(query_3, con = engine)        
print(query_3)

    publisher_id             publisher_name  cnt_books
0            212              Penguin Books         42
1            309                    Vintage         31
2            116   Grand Central Publishing         25
3            217           Penguin Classics         24
4             35                     Bantam         19
5             33           Ballantine Books         19
6             45                    Berkley         17
7             46              Berkley Books         14
8            284         St. Martin's Press         14
9            333  William Morrow Paperbacks         13
10            83                      Delta         13
11           129           Harper Perennial         12
12           133              HarperCollins         12
13           176  Little  Brown and Company         12
14           297                  Tor Books         11
15            31             Back Bay Books         11
16           143                HarperTorch         10
17        

* The top 20 of publishers are displayed in this table
*  Penguin Books released the highest number of books.

## Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [16]:
query_4 = '''
    SELECT    
        books.author_id AS author_id,
        authors.author AS author_name,
        ratings.book_id AS book_id,
        AVG(ratings.rating) AS avg_reating    
    FROM
        ratings
    INNER JOIN
        books ON books.book_id = ratings.book_id
    INNER JOIN
        authors ON authors.author_id = books.author_id    
    GROUP BY
        ratings.book_id, books.author_id, authors.author
    HAVING
        COUNT(ratings.rating_id) > 49
    ORDER BY
        avg_reating DESC
    LIMIT 20;

'''
query_4 = pd.io.sql.read_sql(query_4, con = engine)        
print(query_4)

    author_id                                        author_name  book_id  \
0         236                         J.K. Rowling/Mary GrandPré      302   
1         240                                     J.R.R. Tolkien      722   
2         236                         J.K. Rowling/Mary GrandPré      299   
3         402                  Markus Zusak/Cao Xuân Việt Khương      656   
4         236                         J.K. Rowling/Mary GrandPré      300   
5         376                                  Louisa May Alcott      399   
6         236                         J.K. Rowling/Mary GrandPré      301   
7         240                                     J.R.R. Tolkien      750   
8         498                                       Rick Riordan      779   
9         621                                    William Golding      405   
10        106                                          Dan Brown      696   
11        235                                      J.D. Salinger      673   

We see the top 20 outhors . The highest average rating is 4.41.

## Find the average number of text reviews among users who rated more than 50 books.

In [17]:
query_5 = '''
        SELECT
            sub.username AS username,
            AVG(sub.review_number) AS avg_review_number
        FROM
            (SELECT 
                reviews.username AS username,
                COUNT(reviews.review_id) AS review_number
             FROM
                 reviews
            GROUP BY
                reviews.username
            ) AS sub
        WHERE 
            username IN
            (SELECT
                ratings.username AS username
            FROM 
                ratings
            GROUP BY
                ratings.username
            HAVING
                COUNT(DISTINCT(book_id)) > 50) 
        GROUP BY  
            sub.username
        ORDER BY
            avg_review_number DESC

'''


query_5 = pd.io.sql.read_sql(query_5, con=engine)
print(query_5)


         username  avg_review_number
0     sfitzgerald               28.0
1      martinadam               27.0
2       richard89               26.0
3  jennifermiller               25.0
4          paul88               22.0
5          xdavis               18.0


* There are only 5 users who corespond our requests.
* The higher average number of text review is 28 and the user name is sfitzgerald          

# Conclusion

We went through the topics that were important for us to check out (best authors and publishers, prominent reviewers, ratings) This definitely brought us important initial information that will help us in the further process of developing a new product.